In [ ]:
! pip install pm4py simpy xgboost

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 23.9 MB/s eta 0:00:00
  Created wheel for intervaltree: filename=intervaltree-3.1.0-py2.py3-none-any.whl size=26098 sha256=4b7d99908caeb22167bf6c51670a64ef494f943b147ec564a12e1950447f96f5
  Stored in directory: /root/.cache/pip/wheels/31/d7/d9/eec6891f78cac19a693bd40ecb8365d2f4613318c145ec9816
Successfully built intervaltree


In [ ]:
import random
import numpy as np

In [ ]:
import pm4py
import simpy

/usr/local/lib/python3.11/dist-packages/cudf/utils/gpu_utils.py:75: UserWarning: Failed to dlopen libcuda.so.1
  warnings.warn(str(e))
/usr/local/lib/python3.11/dist-packages/cudf/pandas/__init__.py:64: UserWarning: Function "cuInit" not found
  warnings.warn(str(e))


In [ ]:
from pm4py.objects.log.util import interval_lifecycle
from collections import Counter

In [ ]:
import pandas as pd

In [ ]:
# 1. 이벤트 로그 불러오기
log = pm4py.read_xes('/content/drive/MyDrive/Road_Traffic_Fine_Management_Process.xes') # 데이터프레임 출력

# 2. 케이스별 소요 시간(리드타임) 계산
case_durations = pm4py.get_all_case_durations(log)

# 3. 평균 리드타임 계산 (분 단위로 변환)
average_lead_time_seconds = sum(case_durations) / len(case_durations)
average_lead_time_minutes = average_lead_time_seconds / 60

print(f"총 케이스 수: {len(log)}")
print(f"평균 리드타임: {average_lead_time_minutes:.2f} 분")

parsing log, completed traces ::   0%|          | 0/150370 [00:00<?, ?it/s]

총 케이스 수: 561470
평균 리드타임: 492006.02 분


## Pre-processing

**프로세스에 대한 기본적인 정보 파악**

In [ ]:
# --- 2. 전체 프로세스 성능 지표 ---
print("--- 1. 전체 프로세스 성능 지표 ---")
# 케이스별 소요 시간(리드타임) 계산 (초 단위)
case_durations_sec = pm4py.get_all_case_durations(log)

if case_durations_sec:
    # 분 단위로 변환
    case_durations_min = [d / 60 for d in case_durations_sec]

    avg_lead_time = np.mean(case_durations_min)
    median_lead_time = np.median(case_durations_min)
    min_lead_time = np.min(case_durations_min)
    max_lead_time = np.max(case_durations_min)

    print(f"평균 리드타임: {avg_lead_time:.2f} 분")
    print(f"리드타임 중앙값: {median_lead_time:.2f} 분")
    print(f"최소 리드타임: {min_lead_time:.2f} 분")
    print(f"최대 리드타임: {max_lead_time:.2f} 분")
else:
    print("리드타임을 계산할 수 없습니다.")

# --- 3. 주요 경로 분석 (시나리오 1 관련) ---
print("\n--- 2. 주요 경로 분석 (시나리오 1 관련) ---")
# Directly-Follows Graph (DFG) 생성
dfg, start_activities, end_activities = pm4py.discover_dfg(log)

# 분석 대상 경로 정의
paths_to_analyze = {
    "Create Fine 이후": "Create Fine",
    "Add penalty 이후": "Add penalty",
    "Send Fine 이후": "Send Fine",
    "Insert Fine 이후": "Insert Fine Notification"
}

for desc, source_activity in paths_to_analyze.items():
    # 해당 액티비티에서 나가는 모든 경로와 빈도수 찾기
    outgoing_edges = {k: v for k, v in dfg.items() if k[0] == source_activity}

    if outgoing_edges:
        total_count = sum(outgoing_edges.values())
        print(f"\n[{desc}]")
        print(f"  - 총 {total_count} 건의 분기 발생")
        for (src, dest), count in outgoing_edges.items():
            probability = (count / total_count) * 100
            print(f"    - 경로: {dest:<25} | 빈도: {count:<5} | 확률: {probability:.1f}%")
    else:
        print(f"[{desc}] '{source_activity}'에서 나가는 경로를 찾을 수 없습니다.")


# --- 4. 액티비티 레벨 분석 ---
print("\n--- 3. 액티비티 레벨 분석 ---")
# 로그에 있는 모든 액티비티의 이름과 빈도수 계산
all_activities = pm4py.get_event_attribute_values(log, "concept:name")
activity_counts = Counter(all_activities)

print("액티비티별 발생 빈도 Top 10:")
for activity, count in activity_counts.most_common(10):
    print(f"  - {activity:<30} | {count} 회")

--- 1. 전체 프로세스 성능 지표 ---
평균 리드타임: 492006.02 분
리드타임 중앙값: 285120.00 분
최소 리드타임: 0.00 분
최대 리드타임: 6295680.00 분

--- 2. 주요 경로 분석 (시나리오 1 관련) ---

[Create Fine 이후]
  - 총 150370 건의 분기 발생
    - 경로: Appeal to Judge           | 빈도: 4     | 확률: 0.0%
    - 경로: Insert Date Appeal to Prefecture | 빈도: 22    | 확률: 0.0%
    - 경로: Payment                   | 빈도: 46952 | 확률: 31.2%
    - 경로: Send Fine                 | 빈도: 103392 | 확률: 68.8%

[Add penalty 이후]
  - 총 79860 건의 분기 발생
    - 경로: Appeal to Judge           | 빈도: 80    | 확률: 0.1%
    - 경로: Insert Date Appeal to Prefecture | 빈도: 658   | 확률: 0.8%
    - 경로: Notify Result Appeal to Offender | 빈도: 53    | 확률: 0.1%
    - 경로: Payment                   | 빈도: 18621 | 확률: 23.3%
    - 경로: Receive Result Appeal from Prefecture | 빈도: 351   | 확률: 0.4%
    - 경로: Send Appeal to Prefecture | 빈도: 2915  | 확률: 3.7%
    - 경로: Send for Credit Collection | 빈도: 57182 | 확률: 71.6%

[Send Fine 이후]
  - 총 83232 건의 분기 발생
    - 경로: Appeal to Judge           | 빈도: 10    | 확률: 0.0

**처리 시간 파악(평균치)**

In [ ]:
# 2-1. 케이스 ID와 타임스탬프로 데이터를 정렬하여 이벤트 순서를 보장합니다.
df_sorted = log.sort_values(['case:concept:name', 'time:timestamp'])

# 2-2. 각 케이스 내에서, 바로 이전 이벤트와의 시간 차이를 계산합니다.
df_sorted['time_diff_sec'] = df_sorted.groupby('case:concept:name')['time:timestamp'].diff().dt.total_seconds()

# 2-3. 계산된 시간 차이를 '이전' 액티비티의 처리 시간으로 할당하기 위해 값을 한 행 위로 올립니다.
df_sorted['processing_time_sec'] = df_sorted['time_diff_sec'].shift(-1)

# 2-4. 각 케이스의 마지막 이벤트는 처리 시간을 계산할 수 없으므로 NaN으로 처리합니다.
last_event_indices = df_sorted['case:concept:name'] != df_sorted['case:concept:name'].shift(-1)
df_sorted.loc[last_event_indices, 'processing_time_sec'] = np.nan


# --- 3. 액티비티별 평균 처리 시간 집계 ---
avg_processing_time = df_sorted.groupby('concept:name')['processing_time_sec'].mean().reset_index()

# 보기 쉽게 컬럼 이름 변경 및 분 단위 컬럼 추가
avg_processing_time.rename(columns={'processing_time_sec': 'avg_duration_sec'}, inplace=True)
avg_processing_time['avg_duration_min'] = avg_processing_time['avg_duration_sec'] / 60
avg_processing_time['avg_duration_hour'] = avg_processing_time['avg_duration_min'] / 60

# 시간이 오래 걸리는 순서대로 정렬
sorted_durations = avg_processing_time.sort_values(by='avg_duration_sec', ascending=False)

In [ ]:
# 액티비티별 평균 처리 시간
sorted_durations

,concept:name,avg_duration_sec,avg_duration_min,avg_duration_hour
10,Send for Credit Collection,7.117200e+07,1.186200e+06,19770.000000
0,Add penalty,3.665193e+07,6.108655e+05,10181.090909
5,Notify Result Appeal to Offender,2.051392e+07,3.418987e+05,5698.311111
1,Appeal to Judge,1.470483e+07,2.450805e+05,4084.674584
6,Payment,1.040343e+07,1.733905e+05,2889.842308
2,Create Fine,5.487557e+06,9.145929e+04,1524.321447
4,Insert Fine Notification,4.936812e+06,8.228020e+04,1371.336739
3,Insert Date Appeal to Prefecture,4.588463e+06,7.647438e+04,1274.573066
8,Send Appeal to Prefecture,3.799434e+06,6.332389e+04,1055.398195
7,Receive Result Appeal from Prefecture,3.071858e+06,5.119763e+04,853.293869


**리소스 확인**

In [ ]:
resource_key = "org:resource"
all_resources = pm4py.get_event_attribute_values(log, resource_key)

if not all_resources:
    print(f"\n오류: 로그에서 자원 정보('{resource_key}')를 찾을 수 없습니다. 자원 분석을 진행할 수 없습니다.")
    exit()

print("\n" + "="*50)
print("      As-Is 프로세스 자원(Resource) 분석 (수정본)")
print("="*50 + "\n")


# --- 2. 자원별 전체 작업량 분석 ---
print("--- 1. 자원별 전체 작업량(액티비티 처리 횟수) 분석 ---")
resource_counts = Counter(all_resources)
print(f"전체 프로세스에 참여한 고유 자원 수: {len(resource_counts)} 명")
for resource, count in resource_counts.most_common():
    print(f"  - {resource:<20} | {count} 회 처리")


# --- 3. 주요 액티비티별 담당 자원 분석 (요청사항 반영) ---
print("\n--- 2. 주요 액티비티별 담당 자원 분석 ---")

key_activities = [
    "Create Fine",
    "Add penalty",
    "Send for Credit Collection",
    "Payment"
]

for activity_name in key_activities:
    try:
        filtered_log = pm4py.filter_event_attribute_values(
            log, "concept:name", [activity_name], level="event", retain=True
        )

        print(f"\n* 액티비티: '{activity_name}'")
        if len(filtered_log) > 0:
            # 중복을 포함한 전체 담당 기록을 가져옴
            raw_resources_list = pm4py.get_event_attribute_values(filtered_log, resource_key)

            # 고유한 담당자 목록과 수를 계산
            unique_resources_set = set(raw_resources_list)

            print(f"  - 총 처리 횟수: {len(raw_resources_list)} 회")
            print(f"  - 고유 담당자 수: {len(unique_resources_set)} 명")
            print(f"  - 고유 담당자 목록: {sorted(list(unique_resources_set))}")
            # 요청하신 중복 제거 안 한 전체 리스트 출력
            print(f"  - 전체 담당 기록 (중복 포함): {raw_resources_list}")
        else:
            print(f"  (로그에 해당 액티비티 없음)")

    except Exception as e:
        print(f"'{activity_name}' 분석 중 오류 발생: {e}")


      As-Is 프로세스 자원(Resource) 분석 (수정본)

--- 1. 자원별 전체 작업량(액티비티 처리 횟수) 분석 ---
전체 프로세스에 참여한 고유 자원 수: 148 명
  - 538                  | 8608 회 처리
  - 550                  | 7935 회 처리
  - 541                  | 7356 회 처리
  - 537                  | 6931 회 처리
  - 559                  | 6429 회 처리
  - 557                  | 5436 회 처리
  - 536                  | 4821 회 처리
  - 49                   | 4589 회 처리
  - 561                  | 4043 회 처리
  - 558                  | 3779 회 처리
  - 40                   | 3183 회 처리
  - 53                   | 2913 회 처리
  - 29                   | 2699 회 처리
  - 11                   | 2539 회 처리
  - 28                   | 2278 회 처리
  - 30                   | 2110 회 처리
  - 548                  | 1865 회 처리
  - 546                  | 1779 회 처리
  - 560                  | 1626 회 처리
  - 31                   | 1535 회 처리
  - 63                   | 1522 회 처리
  - 50                   | 1502 회 처리
  - 852                  | 1420 회 처리
  - 563                  | 1391 회 처리
  - 26

In [ ]:
log.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 561470 entries, 0 to 561469
Data columns (total 17 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   amount                230230 non-null  float64       
 1   org:resource          150925 non-null  object        
 2   dismissal             155066 non-null  object        
 3   concept:name          561470 non-null  object        
 4   vehicleClass          150370 non-null  object        
 5   totalPaymentAmount    227971 non-null  float64       
 6   lifecycle:transition  561470 non-null  object        
 7   time:timestamp        561470 non-null  datetime64[ns]
 8   article               150370 non-null  float64       
 9   points                150370 non-null  float64       
 10  case:concept:name     561470 non-null  object        
 11  expense               103987 non-null  float64       
 12  notificationType      79860 non-null   object        
 13 

In [ ]:
# --- 2. Timestamp 컬럼의 타입을 Datetime으로 강제 변환 ---
log['time:timestamp'] = pd.to_datetime(log['time:timestamp'])
print(f"'time:timestamp' 컬럼의 데이터 타입: {log['time:timestamp'].dtype}")


# --- 3. 케이스별 시작 시간 추출 ---
# 케이스 ID('case:concept:name')로 그룹을 묶고, 각 그룹의 첫 번째 타임스탬프를 가져옵니다.
case_start_times = log.groupby('case:concept:name')['time:timestamp'].first().sort_values()


# --- 4. 케이스 간 도착 시간 간격 계산 ---
if len(case_start_times) > 1:
    # .diff()는 바로 이전 값과의 차이를 계산해주는 편리한 pandas 함수입니다.
    # 결과에서 첫 번째 값(NaN)은 제외합니다.
    inter_arrival_times_sec = case_start_times.diff().dt.total_seconds().dropna()

    # --- 5. 평균 도착 시간 간격 계산 ---
    avg_inter_arrival_time_sec = np.mean(inter_arrival_times_sec)

    print("\n--- As-Is 평균 도착 시간 간격 ---")
    print(f"평균 도착 간격: {avg_inter_arrival_time_sec:.2f} 초 ({avg_inter_arrival_time_sec / 60:.2f} 분)")
else:
    print("도착 시간 간격을 계산하기에 케이스가 부족합니다 (최소 2개 필요).")

'time:timestamp' 컬럼의 데이터 타입: datetime64[ns]

--- As-Is 평균 도착 시간 간격 ---
평균 도착 간격: 2825.24 초 (47.09 분)


## 시뮬레이션 As-Is 환경 구축

**1. 액티비티별 평균 처리 시간 & 다음 액티비티 분기 확률 -> PROCESS_MAP_GENERATED**

In [ ]:
from copy import deepcopy

# --- 액티비티 처리 시간 계산 (수동 방식) ---
df_sorted = log.sort_values(['case:concept:name', 'time:timestamp'])
df_sorted['time_diff_sec'] = df_sorted.groupby('case:concept:name')['time:timestamp'].diff().dt.total_seconds()
df_sorted['processing_time_sec'] = df_sorted['time_diff_sec'].shift(-1)
last_event_indices = df_sorted['case:concept:name'] != df_sorted['case:concept:name'].shift(-1)
df_sorted.loc[last_event_indices, 'processing_time_sec'] = np.nan
avg_duration_per_activity = df_sorted.groupby('concept:name')['processing_time_sec'].mean()
print("액티비티별 평균 처리 시간 계산 완료.")

# --- 경로 분기 확률 계산 (DFG + 종료 확률) ---
df_sorted['next_activity'] = df_sorted.groupby('case:concept:name')['concept:name'].shift(-1)
dfg_df = df_sorted.dropna(subset=['next_activity'])
dfg = dfg_df.groupby(['concept:name', 'next_activity']).size()
end_activities = pm4py.discover_dfg(log)[2]
print("경로 분기 정보 (DFG) 계산 완료.")

# --- PROCESS_MAP 자동 생성 (KeyError 해결 로직 적용) ---
source_activities = {x[0] for x in dfg.index}
destination_activities = {x[1] for x in dfg.index}
end_activity_names = set(end_activities.keys())
# DFG의 모든 출발지, 목적지, 그리고 종료 액티비티를 포함하여 전체 목록 생성
all_activities_in_log = sorted(list(source_activities | destination_activities | end_activity_names))

PROCESS_MAP_GENERATED = {}

for activity_name in all_activities_in_log:
    outgoing_edges = dfg.get(activity_name, {})
    outgoing_count = sum(outgoing_edges.values) if not outgoing_edges.empty else 0
    end_count = end_activities.get(activity_name, 0)
    total_exits = outgoing_count + end_count

    next_activity_logic = {}
    if total_exits > 0:
        if not outgoing_edges.empty:
            for dest_activity, count in outgoing_edges.items():
                next_activity_logic[dest_activity] = count / total_exits
        if end_count > 0:
            next_activity_logic['END'] = end_count / total_exits
    else:
        next_activity_logic['END'] = 1.0

    if activity_name == "Create Fine":
      PROCESS_MAP_GENERATED[activity_name] = {
        'processing_time': avg_duration_per_activity.get(activity_name, 0),
        'resource_pool': 'staff',
        'next_activity_logic': next_activity_logic
       }
    else:
      PROCESS_MAP_GENERATED[activity_name] = {
        'processing_time': avg_duration_per_activity.get(activity_name, 0),
        'next_activity_logic': next_activity_logic
       }

액티비티별 평균 처리 시간 계산 완료.
경로 분기 정보 (DFG) 계산 완료.


In [ ]:
PROCESS_MAP_GENERATED

{'Add penalty': {'processing_time': np.float64(36651927.27272727),
  'next_activity_logic': {'Appeal to Judge': np.float64(0.0010017530678687703),
   'Insert Date Appeal to Prefecture': np.float64(0.008239418983220637),
   'Notify Result Appeal to Offender': np.float64(0.0006636614074630603),
   'Payment': np.float64(0.23317054845980467),
   'Receive Result Appeal from Prefecture': np.float64(0.00439519158527423),
   'Send Appeal to Prefecture': np.float64(0.03650137741046832),
   'Send for Credit Collection': np.float64(0.7160280490859003)}},
 'Appeal to Judge': {'processing_time': np.float64(14704828.503562946),
  'next_activity_logic': {'Add penalty': np.float64(0.5063063063063064),
   'Insert Date Appeal to Prefecture': np.float64(0.02702702702702703),
   'Notify Result Appeal to Offender': np.float64(0.016216216216216217),
   'Payment': np.float64(0.12612612612612611),
   'Receive Result Appeal from Prefecture': np.float64(0.0018018018018018018),
   'Send Appeal to Prefecture': np

In [ ]:
# Part 2. 시뮬레이션 환경 설정 및 실행

# --- 시뮬레이션 파라미터 ---
RANDOM_SEED = 42
SIM_TIME = 86400 * 180  # 충분히 길게 설정 (180일)
AVG_ARRIVAL_TIME = 2825.24
RESOURCES = {'staff': {'capacity': 148}}

# --- Part 1에서 생성된 맵을 As-Is 기준으로 사용 ---
PROCESS_MAP_AS_IS = PROCESS_MAP_GENERATED

# --- 결과 저장을 위한 전역 변수 ---
results_lead_time = []
# ... 기타 카운터 ...

# --- SimPy 클래스 및 함수 정의 (이전과 동일) ---
class Process:
    def __init__(self, env, case_id, resources, process_map):
        self.env = env
        self.case_id = case_id
        self.resources = resources
        self.process_map = process_map
        self.arrival_time = self.env.now
        self.action = env.process(self.run_workflow())

    def run_workflow(self):
        current_activity_name = 'Create Fine'
        while current_activity_name != 'END':
            # print(f"시간: {self.env.now:9.2f}, 케이스: {self.case_id:<4}, 현재 액티비티: {current_activity_name}")
            activity_details = self.process_map.get(current_activity_name)
            if not activity_details:
                # print(f"오류: 맵에 '{current_activity_name}' 없음. 종료.")
                break
            yield self.env.process(self.execute_task(current_activity_name, activity_details))
            next_logic = activity_details.get('next_activity_logic', {})
            rand_val = random.random()
            cumulative_prob, next_activity_name = 0.0, 'END'
            if next_logic:
                for next_name, prob in next_logic.items():
                    cumulative_prob += prob
                    if rand_val < cumulative_prob:
                        next_activity_name = next_name
                        break
            current_activity_name = next_activity_name
        if current_activity_name == 'END':
            # print(f"시간: {self.env.now:9.2f}, 케이스: {self.case_id:<4}, 상태: ★완료★")
            lead_time = self.env.now - self.arrival_time
            results_lead_time.append(lead_time)

    def execute_task(self, task_name, details):
        processing_time = details.get('processing_time', 0)
        resource_name = details.get('resource_pool', 'default')
        resource_to_use = self.resources.get(resource_name)
        if resource_to_use:
            with resource_to_use.request() as req:
                yield req
                yield self.env.timeout(processing_time)
        else:
            yield self.env.timeout(processing_time)

def source(env, resources, process_map):
    case_id = 0
    while True:
        yield env.timeout(random.expovariate(1.0 / AVG_ARRIVAL_TIME))
        case_id += 1
        Process(env, case_id, resources, process_map)

In [ ]:
# --- 시뮬레이션 실행 ---
random.seed(RANDOM_SEED)
env = simpy.Environment()
simpy_resources = {name: simpy.Resource(env, **props) for name, props in RESOURCES.items()}
env.process(source(env, simpy_resources, PROCESS_MAP_AS_IS))
env.run(until=SIM_TIME)

# --- 결과 분석 ---
print("\n--- 최종 시뮬레이션 결과 ---")
if results_lead_time:
  total_cases = len(results_lead_time)
  avg_lead_time_sec = np.mean(results_lead_time)
  print(f"총 처리된 케이스 수: {total_cases}")
  print(f"평균 리드타임: {avg_lead_time_sec / 3600:.2f} 시간")
else:
  print("처리된 케이스가 없습니다.")


--- 최종 시뮬레이션 결과 ---
총 처리된 케이스 수: 38
평균 리드타임: 2562.71 시간


In [ ]:
# # --- 2. 전이 시간 계산을 위한 데이터 정렬 및 가공 (실제 적용 X)

# # 케이스 ID와 타임스탬프로 데이터를 정렬하여 이벤트 순서를 보장
# df_sorted = log.sort_values(['case:concept:name', 'time:timestamp'])

# # 현재 액티비티(source)와 바로 다음 액티비티(target)를 같은 행에 놓기 위해 shift 사용
# df_sorted['source_activity'] = df_sorted['concept:name']
# df_sorted['target_activity'] = df_sorted.groupby('case:concept:name')['concept:name'].shift(-1)

# # 현재 타임스탬프와 바로 다음 타임스탬프의 차이를 계산하여 전이 시간으로 정의
# # .diff(-1)는 현재 행과 다음 행의 차이를 계산합니다. .abs()로 양수 변환.
# df_sorted['transition_time_sec'] = df_sorted.groupby('case:concept:name')['time:timestamp'].diff(-1).abs().dt.total_seconds()

# # 다음 액티비티가 없는 행(케이스의 마지막)은 계산에서 제외
# transitions_df = df_sorted.dropna(subset=['target_activity'])


# # --- 3. 전이별 평균 시간 계산 ---

# # 'source_activity'와 'target_activity' 쌍으로 그룹을 묶어,
# # 각 그룹의 'transition_time_sec'의 평균을 계산합니다.
# avg_transition_times = transitions_df.groupby(['source_activity', 'target_activity'])['transition_time_sec'].mean().reset_index()

# # 보기 쉽게 분 단위 컬럼 추가 및 정렬
# avg_transition_times['transition_time_min'] = avg_transition_times['transition_time_sec'] / 60
# sorted_transitions = avg_transition_times.sort_values(by='transition_time_sec', ascending=False)


# # --- 4. 결과 출력 ---
# print("      전이(Transition)별 평균 소요 시간")
# print(sorted_transitions.to_string(index=False))

## Scenarios

### To-be Scenario 1

In [ ]:
# 1. 확률의 합계를 1로 자동 보정해주는 헬퍼(helper) 함수를 정의합니다.
def normalize_probabilities(prob_dict):
    """
    입력된 딕셔너리의 값(확률)들의 합이 1이 되도록 보정합니다.
    예: {'A': 0.8, 'B': 0.4} -> {'A': 0.666, 'B': 0.333}
    """
    total = sum(prob_dict.values())
    if total == 0:
        return prob_dict

    # 각 확률을 전체 합으로 나누어 정규화
    normalized_dict = {key: value / total for key, value in prob_dict.items()}
    return normalized_dict

# 2. As-Is 맵을 깊은 복사하여 시나리오 1의 맵을 만듭니다.
PROCESS_MAP_SCENARIO_1 = deepcopy(PROCESS_MAP_AS_IS)
s1_map = PROCESS_MAP_SCENARIO_1

# 3. 시나리오에 맞게 특정 경로의 확률 값을 '일단' 변경합니다.
#    (이 단계에서는 확률의 총합이 1을 넘을 수 있습니다)

# 시나리오: ('Create Fine' -> 'Payment') 확률 25% 증가
# As-Is 확률: 0.312 -> To-Be 목표 확률: 0.312 * 1.25 = 0.39
s1_map['Create Fine']['next_activity_logic']['Payment'] *= 1.25

# 시나리오: ('Add penalty' -> 'Payment') 확률 25% 증가
# As-Is 확률: 0.233 -> To-Be 목표 확률: 0.233 * 1.25 = 0.291
s1_map['Add penalty']['next_activity_logic']['Payment'] *= 1.25

# 시나리오: ('Add penalty' -> 'Send for Credit Collection') 확률 25% 감소
s1_map['Add penalty']['next_activity_logic']['Send for Credit Collection'] *= (1 - 0.25)


# 4. 변경된 확률들을 대상으로 '자동 보정 함수'를 호출하여 총합을 1로 맞춥니다.
s1_map['Create Fine']['next_activity_logic'] = \
    normalize_probabilities(s1_map['Create Fine']['next_activity_logic'])

s1_map['Add penalty']['next_activity_logic'] = \
    normalize_probabilities(s1_map['Add penalty']['next_activity_logic'])


# (선택사항) 보정된 확률이 잘 적용되었는지 확인
print("--- 보정된 'Create Fine' 이후 확률 ---")
print(s1_map['Create Fine']['next_activity_logic'])
print("확률 총합:", sum(s1_map['Create Fine']['next_activity_logic'].values()))

print("\n--- 보정된 'Add penalty' 이후 확률 ---")
print(s1_map['Add penalty']['next_activity_logic'])
print("확률 총합:", sum(s1_map['Add penalty']['next_activity_logic'].values()))

--- 보정된 'Create Fine' 이후 확률 ---
{'Appeal to Judge': np.float64(2.4674908085967378e-05), 'Insert Date Appeal to Prefecture': np.float64(0.0001357119944728206), 'Payment': np.float64(0.36204258889135643), 'Send Fine': np.float64(0.6377970242060848)}
확률 총합: 1.0

--- 보정된 'Add penalty' 이후 확률 ---
{'Appeal to Judge': np.float64(0.0011392806154963523), 'Insert Date Appeal to Prefecture': np.float64(0.0093705830624575), 'Notify Result Appeal to Offender': np.float64(0.0007547734077663334), 'Payment': np.float64(0.33147725533058714), 'Receive Result Appeal from Prefecture': np.float64(0.0049985937004902465), 'Send Appeal to Prefecture': np.float64(0.04151253742714834), 'Send for Credit Collection': np.float64(0.610746976456054)}
확률 총합: 1.0


In [ ]:
# --- 시뮬레이션 실행 ---
random.seed(RANDOM_SEED)
env = simpy.Environment()
simpy_resources = {name: simpy.Resource(env, **props) for name, props in RESOURCES.items()}
env.process(source(env, simpy_resources, s1_map))
env.run(until=SIM_TIME)

# --- 결과 분석 ---
print("\n--- 최종 시뮬레이션 결과 ---")
if results_lead_time:
  total_cases = len(results_lead_time)
  avg_lead_time_sec = np.mean(results_lead_time)
  print(f"총 처리된 케이스 수: {total_cases}")
  print(f"평균 리드타임: {avg_lead_time_sec / 3600:.2f} 시간")
else:
  print("처리된 케이스가 없습니다.")


--- 최종 시뮬레이션 결과 ---
총 처리된 케이스 수: 67
평균 리드타임: 2516.27 시간


In [ ]:
# As-Is 맵을 깊은 복사하여 시나리오 1(sax4bpm버전) 의 맵을 만듭니다.
PROCESS_MAP_SCENARIO_2 = deepcopy(PROCESS_MAP_AS_IS)
s2_map = PROCESS_MAP_SCENARIO_2

# --- 규칙 1: ('Create Fine' -> 'Payment') 확률 20% 증가 (15-25%의 중앙값) ---
# 이 액티비티의 다른 경로들의 확률은 원래 비율대로 자동 조정됩니다.
print("--- 1. 'Create Fine' 경로 확률 조정 ---")
cf_logic = s2_map['Create Fine']['next_activity_logic']
print("Before:", cf_logic)
if 'Payment' in cf_logic:
    cf_logic['Payment'] *= 1.20 # 20% 증가
    s2_map['Create Fine']['next_activity_logic'] = normalize_probabilities(cf_logic)
    print("After: ", s2_map['Create Fine']['next_activity_logic'])
    print("Sum:   ", sum(s2_map['Create Fine']['next_activity_logic'].values()))
else:
    print("'Create Fine'에서 'Payment'로 가는 경로가 없어 조정하지 않습니다.")

# --- 규칙 2: ('Send Fine' -> 'Payment') 확률 25% 증가 (20-30%의 중앙값) ---
print("\n--- 2. 'Send Fine' 경로 확률 조정 ---")
sf_logic = s2_map['Send Fine']['next_activity_logic']
print("Before:", sf_logic)
if 'Payment' in sf_logic:
    sf_logic['Payment'] *= 1.25 # 25% 증가
    s2_map['Send Fine']['next_activity_logic'] = normalize_probabilities(sf_logic)
    print("After: ", s2_map['Send Fine']['next_activity_logic'])
    print("Sum:   ", sum(s2_map['Send Fine']['next_activity_logic'].values()))
else:
    print("'Send Fine'에서 'Payment'로 가는 경로가 없어 조정하지 않습니다.")


# --- 규칙 3 & 4: 'Insert Fine Notification' 경로 조정 (특수 로직) ---
# 'Payment'가 늘어난 '절대량'만큼 'Add penalty'에서 빼주는 방식
print("\n--- 3. 'Insert Fine Notification' 경로 확률 조정 ---")
ifn_logic = s2_map['Insert Fine Notification']['next_activity_logic']
print("Before:", ifn_logic)

if 'Payment' in ifn_logic and 'Add penalty' in ifn_logic:
    # 'Payment' 확률이 얼마나 '절대적'으로 증가했는지 계산
    original_payment_prob = ifn_logic['Payment']
    # 30% 증가 (25-35%의 중앙값)
    new_payment_prob = original_payment_prob * 1.30
    increase_amount = new_payment_prob - original_payment_prob

    # 'Payment' 확률을 업데이트
    ifn_logic['Payment'] = new_payment_prob

    # 'Add penalty' 확률에서 그 증가량만큼 정확히 빼줌
    ifn_logic['Add penalty'] -= increase_amount

    # 만약의 경우를 대비해 음수가 되지 않도록 보정
    if ifn_logic['Add penalty'] < 0:
        print("Warning: 'Add penalty' 확률이 음수가 되어 0으로 조정합니다.")
        # 음수가 된 만큼 다른 경로에서 추가로 빼주거나 하는 로직이 필요하지만, 여기서는 0으로만 처리
        ifn_logic['Add penalty'] = 0
        # 이 경우, 합계가 1이 안 맞을 수 있으므로 다시 normalize
        s2_map['Insert Fine Notification']['next_activity_logic'] = normalize_probabilities(ifn_logic)

    print("After: ", ifn_logic)
    print("Sum:   ", sum(ifn_logic.values()))
else:
    print("해당 경로가 없어 조정하지 않습니다.")

--- 1. 'Create Fine' 경로 확률 조정 ---
Before: {'Appeal to Judge': np.float64(2.660105074150429e-05), 'Insert Date Appeal to Prefecture': np.float64(0.0001463057790782736), 'Payment': np.float64(0.31224313360377737), 'Send Fine': np.float64(0.6875839595664028)}
After:  {'Appeal to Judge': np.float64(2.5037493646735984e-05), 'Insert Date Appeal to Prefecture': np.float64(0.00013770621505704793), 'Payment': np.float64(0.3526681205104644), 'Send Fine': np.float64(0.6471691357808317)}
Sum:    0.9999999999999999

--- 2. 'Send Fine' 경로 확률 조정 ---
Before: {'Appeal to Judge': np.float64(9.616586688720705e-05), 'Insert Date Appeal to Prefecture': np.float64(0.0015482704568840337), 'Insert Fine Notification': np.float64(0.7669901045322973), 'Payment': np.float64(0.03173473607277833), 'Send Appeal to Prefecture': np.float64(3.8466346754882824e-05), 'END': np.float64(0.19959225672439823)}
After:  {'Appeal to Judge': np.float64(9.540892264244552e-05), 'Insert Date Appeal to Prefecture': np.float64(0.0015

In [ ]:
# --- 시뮬레이션 실행 ---
random.seed(RANDOM_SEED)
env = simpy.Environment()
simpy_resources = {name: simpy.Resource(env, **props) for name, props in RESOURCES.items()}
env.process(source(env, simpy_resources, s2_map))
env.run(until=SIM_TIME)

# --- 결과 분석 ---
print("\n--- 최종 시뮬레이션 결과 ---")
if results_lead_time:
  total_cases = len(results_lead_time)
  avg_lead_time_sec = np.mean(results_lead_time)
  print(f"총 처리된 케이스 수: {total_cases}")
  print(f"평균 리드타임: {avg_lead_time_sec / 3600:.2f} 시간")
else:
  print("처리된 케이스가 없습니다.")


--- 최종 시뮬레이션 결과 ---
총 처리된 케이스 수: 105
평균 리드타임: 2586.23 시간


### To-be Scenario 2

In [ ]:
# --- To-Be 시나리오 2---
print("\n" + "="*50)
print("      To-Be 시나리오 2 (pm4py 버전) 생성 시작: 처리 시간 단축")
print("="*50 + "\n")

# As-Is 맵을 깊은 복사하여 시나리오 3의 맵을 만듭니다.
PROCESS_MAP_SCENARIO_3 = deepcopy(PROCESS_MAP_AS_IS)
s3_map = PROCESS_MAP_SCENARIO_3

# 처리 시간을 단축할 액티비티 목록
activities_to_speed_up = [
    'Insert Date Appeal to Prefecture',
    'Send Appeal to Prefecture',
    'Receive Result Appeal from Prefecture',
    'Notify Result Appeal to Offender',
    'Appeal to Judge'
]

# 단축률 (30-50%의 중앙값)
reduction_rate = 0.40 # 40% 단축

print(f"--- 항소 관련 액티비티 처리 시간을 {reduction_rate:.0%} 단축합니다 ---")

for activity_name in activities_to_speed_up:
    # 해당 액티비티가 프로세스 맵에 있는지 확인
    if activity_name in s3_map:
        original_time = s3_map[activity_name]['processing_time']

        # 처리 시간을 40% 단축 (원래 시간의 60%가 됨)
        new_time = original_time * (1 - reduction_rate)
        s3_map[activity_name]['processing_time'] = new_time

        print(f"\n* '{activity_name}' 처리 시간 변경:")
        print(f"  Before: {original_time:.2f} 초")
        print(f"  After:  {new_time:.2f} 초")
    else:
        print(f"\n* '{activity_name}'이(가) 프로세스 맵에 없어 변경하지 않습니다.")

# --- 시뮬레이션 실행 ---
random.seed(RANDOM_SEED)
env = simpy.Environment()
simpy_resources = {name: simpy.Resource(env, **props) for name, props in RESOURCES.items()}
env.process(source(env, simpy_resources, s3_map))
env.run(until=SIM_TIME)

# --- 결과 분석 ---
print("\n--- 최종 시뮬레이션 결과 ---")
if results_lead_time:
  total_cases = len(results_lead_time)
  avg_lead_time_sec = np.mean(results_lead_time)
  print(f"총 처리된 케이스 수: {total_cases}")
  print(f"평균 리드타임: {avg_lead_time_sec / 3600:.2f} 시간")
else:
  print("처리된 케이스가 없습니다.")


      To-Be 시나리오 2 (pm4py 버전) 생성 시작: 처리 시간 단축

--- 항소 관련 액티비티 처리 시간을 40% 단축합니다 ---

* 'Insert Date Appeal to Prefecture' 처리 시간 변경:
  Before: 4588463.04 초
  After:  2753077.82 초

* 'Send Appeal to Prefecture' 처리 시간 변경:
  Before: 3799433.50 초
  After:  2279660.10 초

* 'Receive Result Appeal from Prefecture' 처리 시간 변경:
  Before: 3071857.93 초
  After:  1843114.76 초

* 'Notify Result Appeal to Offender' 처리 시간 변경:
  Before: 20513920.00 초
  After:  12308352.00 초

* 'Appeal to Judge' 처리 시간 변경:
  Before: 14704828.50 초
  After:  8822897.10 초

--- 최종 시뮬레이션 결과 ---
총 처리된 케이스 수: 142
평균 리드타임: 2574.37 시간


In [ ]:
# --- To-Be 시나리오 3 ---
print("\n" + "="*50)
print("      To-Be 시나리오 3 (pm4py 버전) 생성 시작")
print("="*50 + "\n")

# As-Is 맵을 깊은 복사하여 복합 시나리오의 맵을 만듭니다.
PROCESS_MAP_COMPLEX = deepcopy(PROCESS_MAP_AS_IS)
s_complex_map = PROCESS_MAP_COMPLEX

# 확률 보정을 위한 헬퍼 함수 (필요 시 사용)
def normalize_probabilities(prob_dict):
    total = sum(prob_dict.values())
    if total == 0: return prob_dict
    return {key: value / total for key, value in prob_dict.items()}

# --- 규칙 1 & 2: 'Add penalty' 경로 조정 ---
# 'Payment'가 늘어난 절대량만큼 'Send for Credit Collection'에서 빼주는 방식
print("--- 1. 'Add penalty' 경로 확률 조정 ---")
ap_logic = s_complex_map['Add penalty']['next_activity_logic']
print("Before:", ap_logic)

if 'Payment' in ap_logic and 'Send for Credit Collection' in ap_logic:
    # 'Payment' 확률이 얼마나 '절대적'으로 증가했는지 계산
    original_payment_prob = ap_logic['Payment']
    # 27.5% 증가 (20-35%의 중앙값)
    new_payment_prob = original_payment_prob * 1.275
    increase_amount = new_payment_prob - original_payment_prob

    # 'Payment' 확률 업데이트
    ap_logic['Payment'] = new_payment_prob
    # 'Send for Credit Collection' 확률에서 그 증가량만큼 정확히 빼줌
    ap_logic['Send for Credit Collection'] -= increase_amount

    # 음수가 되지 않도록 보정
    if ap_logic['Send for Credit Collection'] < 0:
        ap_logic['Send for Credit Collection'] = 0

    # 이 특별 케이스에서는 다른 경로를 건드리지 않았으므로, normalize가 필요 없음
    # (단, 음수 보정 시에는 필요할 수 있음)
    print("After: ", ap_logic)
    print("Sum:   ", sum(ap_logic.values()))
else:
    print("해당 경로가 없어 조정하지 않습니다.")

# --- 규칙 3: 'Payment' -> 'Add penalty' 루프 감소 ---
print("\n--- 2. 'Payment' 경로 확률 조정 ---")
p_logic = s_complex_map['Payment']['next_activity_logic']
print("Before:", p_logic)

if 'Add penalty' in p_logic:
    # 먼저 'Add penalty'로 가는 확률을 15% 감소 (10-20%의 중앙값)
    p_logic['Add penalty'] *= (1 - 0.15)

    # 이제 나머지 확률들의 합을 1로 만들기 위해 보정
    # 'Add penalty'를 제외한 나머지 경로들의 현재 확률 합 계산
    fixed_prob = p_logic['Add penalty']
    current_other_sum = sum(prob for key, prob in p_logic.items() if key != 'Add penalty')

    # 목표로 해야 할 나머지 확률들의 합 (1에서 고정된 확률을 뺌)
    target_other_sum = 1 - fixed_prob

    # 보정 계수 계산
    if current_other_sum > 0:
        renorm_factor = target_other_sum / current_other_sum
        # 'Add penalty'를 제외한 모든 경로에 보정 계수 적용
        for key in p_logic:
            if key != 'Add penalty':
                p_logic[key] *= renorm_factor

    print("After: ", p_logic)
    print("Sum:   ", sum(p_logic.values()))
else:
    print("해당 경로가 없어 조정하지 않습니다.")


      To-Be 시나리오 3 (pm4py 버전) 생성 시작

--- 1. 'Add penalty' 경로 확률 조정 ---
Before: {'Appeal to Judge': np.float64(0.0010017530678687703), 'Insert Date Appeal to Prefecture': np.float64(0.008239418983220637), 'Notify Result Appeal to Offender': np.float64(0.0006636614074630603), 'Payment': np.float64(0.23317054845980467), 'Receive Result Appeal from Prefecture': np.float64(0.00439519158527423), 'Send Appeal to Prefecture': np.float64(0.03650137741046832), 'Send for Credit Collection': np.float64(0.7160280490859003)}
After:  {'Appeal to Judge': np.float64(0.0010017530678687703), 'Insert Date Appeal to Prefecture': np.float64(0.008239418983220637), 'Notify Result Appeal to Offender': np.float64(0.0006636614074630603), 'Payment': np.float64(0.2972924492862509), 'Receive Result Appeal from Prefecture': np.float64(0.00439519158527423), 'Send Appeal to Prefecture': np.float64(0.03650137741046832), 'Send for Credit Collection': np.float64(0.6519061482594541)}
Sum:    1.0

--- 2. 'Payment' 경로 확률 조

In [ ]:
# --- To-Be 시나리오 3: 항소 관련 액티비티 처리 시간 단축 ---
print("\n" + "="*50)
print("      To-Be 시나리오 3 생성 (pm4py 버전) 시작")
print("="*50 + "\n")

# As-Is 맵을 깊은 복사하여 시나리오 5의 맵을 만듭니다.
PROCESS_MAP_SCENARIO_5 = deepcopy(PROCESS_MAP_AS_IS)
s5_map = PROCESS_MAP_SCENARIO_5

# 처리 시간을 단축할 액티비티 목록
activities_to_speed_up = [
    'Insert Fine Notification',
    'Add penalty'
]

# 단축률 (30-50%의 중앙값)
reduction_rate = 0.30 # 40% 단축

print(f"--- 항소 관련 액티비티 처리 시간을 {reduction_rate:.0%} 단축합니다 ---")

for activity_name in activities_to_speed_up:
    # 해당 액티비티가 프로세스 맵에 있는지 확인
    if activity_name in s5_map:
        original_time = s5_map[activity_name]['processing_time']

        # 처리 시간을 40% 단축 (원래 시간의 60%가 됨)
        new_time = original_time * (1 - reduction_rate)
        s3_map[activity_name]['processing_time'] = new_time

        print(f"\n* '{activity_name}' 처리 시간 변경:")
        print(f"  Before: {original_time:.2f} 초")
        print(f"  After:  {new_time:.2f} 초")
    else:
        print(f"\n* '{activity_name}'이(가) 프로세스 맵에 없어 변경하지 않습니다.")

# --- 시뮬레이션 실행 ---
random.seed(RANDOM_SEED)
env = simpy.Environment()
simpy_resources = {name: simpy.Resource(env, **props) for name, props in RESOURCES.items()}
env.process(source(env, simpy_resources, s3_map))
env.run(until=SIM_TIME)

# --- 결과 분석 ---
print("\n--- 최종 시뮬레이션 결과 ---")
if results_lead_time:
  total_cases = len(results_lead_time)
  avg_lead_time_sec = np.mean(results_lead_time)
  print(f"총 처리된 케이스 수: {total_cases}")
  print(f"평균 리드타임: {avg_lead_time_sec / 3600:.2f} 시간")
else:
  print("처리된 케이스가 없습니다.")


      To-Be 시나리오 3 생성 (pm4py 버전) 시작

--- 항소 관련 액티비티 처리 시간을 30% 단축합니다 ---

* 'Insert Fine Notification' 처리 시간 변경:
  Before: 4936812.26 초
  After:  3455768.58 초

* 'Add penalty' 처리 시간 변경:
  Before: 36651927.27 초
  After:  25656349.09 초

--- 최종 시뮬레이션 결과 ---
총 처리된 케이스 수: 178
평균 리드타임: 2562.80 시간


In [ ]:
# --- 시뮬레이션 실행 ---
random.seed(RANDOM_SEED)
env = simpy.Environment()
simpy_resources = {name: simpy.Resource(env, **props) for name, props in RESOURCES.items()}
env.process(source(env, simpy_resources, s_complex_map))
env.run(until=SIM_TIME)

# --- 결과 분석 ---
print("\n--- 최종 시뮬레이션 결과 ---")
if results_lead_time:
  total_cases = len(results_lead_time)
  avg_lead_time_sec = np.mean(results_lead_time)
  print(f"총 처리된 케이스 수: {total_cases}")
  print(f"평균 리드타임: {avg_lead_time_sec / 3600:.2f} 시간")
else:
  print("처리된 케이스가 없습니다.")


--- 최종 시뮬레이션 결과 ---
총 처리된 케이스 수: 216
평균 리드타임: 2562.78 시간


In [ ]:
PROCESS_MAP_SCENARIO_6 = deepcopy(PROCESS_MAP_AS_IS)
s6_map = PROCESS_MAP_SCENARIO_6

# --- 규칙 1, 2, 3: 'Notify Result Appeal to Offender' 경로 조정 ---
print("--- 1. 'Notify Result Appeal to Offender' 경로 확률 조정 ---")
nrao_logic = s6_map['Notify Result Appeal to Offender']['next_activity_logic']
print("Before:", nrao_logic)

# 규칙 적용 (중앙값 사용)
if 'Payment' in nrao_logic:
    nrao_logic['Payment'] *= 1.40 # 40% 증가 (30-50%의 중앙값)
if 'Send for Credit Collection' in nrao_logic:
    nrao_logic['Send for Credit Collection'] *= (1 - 0.25) # 25% 감소 (20-30%의 중앙값)
if 'Appeal to Judge' in nrao_logic:
    nrao_logic['Appeal to Judge'] *= (1 - 0.20) # 20% 감소 (15-25%의 중앙값)

# 변경된 확률들의 총합을 1로 보정
s6_map['Notify Result Appeal to Offender']['next_activity_logic'] = normalize_probabilities(nrao_logic)
print("After: ", s6_map['Notify Result Appeal to Offender']['next_activity_logic'])
print("Sum:   ", sum(s6_map['Notify Result Appeal to Offender']['next_activity_logic'].values()))


# --- 규칙 4: 'Appeal to Judge' 경로 조정 ---
print("\n--- 2. 'Appeal to Judge' 경로 확률 조정 ---")
aj_logic = s6_map['Appeal to Judge']['next_activity_logic']
print("Before:", aj_logic)

if 'Add penalty' in aj_logic:
    aj_logic['Add penalty'] *= (1 - 0.25) # 25% 감소 (20-30%의 중앙값)
    # 총합을 1로 보정
    s6_map['Appeal to Judge']['next_activity_logic'] = normalize_probabilities(aj_logic)
    print("After: ", s6_map['Appeal to Judge']['next_activity_logic'])
    print("Sum:   ", sum(s6_map['Appeal to Judge']['next_activity_logic'].values()))
else:
    print("해당 경로가 없어 조정하지 않습니다.")


# --- 규칙 5: 'Send Appeal to Prefecture' 경로 조정 ---
print("\n--- 3. 'Send Appeal to Prefecture' 경로 확률 조정 ---")
sap_logic = s6_map['Send Appeal to Prefecture']['next_activity_logic']
print("Before:", sap_logic)

if 'Add penalty' in sap_logic:
    sap_logic['Add penalty'] *= (1 - 0.20) # 20% 감소 (15-25%의 중앙값)
    # 총합을 1로 보정
    s6_map['Send Appeal to Prefecture']['next_activity_logic'] = normalize_probabilities(sap_logic)
    print("After: ", s6_map['Send Appeal to Prefecture']['next_activity_logic'])
    print("Sum:   ", sum(s6_map['Send Appeal to Prefecture']['next_activity_logic'].values()))
else:
    print("해당 경로가 없어 조정하지 않습니다.")

--- 1. 'Notify Result Appeal to Offender' 경로 확률 조정 ---
Before: {'Add penalty': np.float64(0.013392857142857142), 'Appeal to Judge': np.float64(0.16183035714285715), 'Payment': np.float64(0.43638392857142855), 'Receive Result Appeal from Prefecture': np.float64(0.002232142857142857), 'Send Appeal to Prefecture': np.float64(0.0033482142857142855), 'Send for Credit Collection': np.float64(0.28683035714285715), 'END': np.float64(0.09598214285714286)}
After:  {'Add penalty': np.float64(0.012511077516551112), 'Appeal to Judge': np.float64(0.12094041599332743), 'Payment': np.float64(0.5707136527133398), 'Receive Result Appeal from Prefecture': np.float64(0.002085179586091852), 'Send Appeal to Prefecture': np.float64(0.003127769379137778), 'Send for Credit Collection': np.float64(0.20095918260960224), 'END': np.float64(0.08966272220194965)}
Sum:    0.9999999999999998

--- 2. 'Appeal to Judge' 경로 확률 조정 ---
Before: {'Add penalty': np.float64(0.5063063063063064), 'Insert Date Appeal to Prefecture

In [ ]:
# --- 시뮬레이션 실행 ---
random.seed(RANDOM_SEED)
env = simpy.Environment()
simpy_resources = {name: simpy.Resource(env, **props) for name, props in RESOURCES.items()}
env.process(source(env, simpy_resources, s6_map))
env.run(until=SIM_TIME)

# --- 결과 분석 ---
print("\n--- 최종 시뮬레이션 결과 ---")
if results_lead_time:
  total_cases = len(results_lead_time)
  avg_lead_time_sec = np.mean(results_lead_time)
  print(f"총 처리된 케이스 수: {total_cases}")
  print(f"평균 리드타임: {avg_lead_time_sec / 3600:.2f} 시간")
else:
  print("처리된 케이스가 없습니다.")


--- 최종 시뮬레이션 결과 ---
총 처리된 케이스 수: 254
평균 리드타임: 2562.77 시간
